# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [3]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [4]:
# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


#checking teachers test accuracy
X_test = X_test.reshape(10000, 28, 28, 1)

X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255
preds=teacher.predict(X_test)

numbers=[0,1,2,3,4,5,6,7,8,9]
preds2=[]
for p in preds:
    preds2.append(numbers[np.argmax(p)])
preds2=np.array(preds2)

print (y_test)
print (preds2)
print (np.sum(y_test==preds2))

[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
9899


### Evaluation 

In [5]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
X_test = X_test.reshape(10000, 28, 28, 1)

In [6]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0541 - accuracy: 0.9869


[0.03862640634179115, 0.9898999929428101]

### Dense Vector split

In [27]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

# 4 Students case
# ---------------------------------------------
s1Train=train_dense[:,:64]
s2Train=train_dense[:,64:128]
s3Train=train_dense[:,128:192]
s4Train=train_dense[:,192:]

s1Val=val_dense[:,:64]
s2Val=val_dense[:,64:128]
s3Val=val_dense[:,128:192]
s4Val=val_dense[:,192:]

### Student

In [28]:
def define_student(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(8, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(64,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])
    
    return student1



You can initialize multiple instances here 

In [29]:
s1=define_student("s1")
s2=define_student('s2')
s3=define_student("s3")
s4=define_student('s4')


In [30]:
s1.fit(X_train, s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=(X_val, s1Val))
s2.fit(X_train, s2Train,
          batch_size=batch_size,
          epochs=10,verbose=1, validation_data=(X_val, s2Val))
s3.fit(X_train, s3Train,
          batch_size=batch_size,
          epochs=10,verbose=1, validation_data=(X_val, s3Val))
s4.fit(X_train, s4Train,
          batch_size=batch_size,
          epochs=10,verbose=1, validation_data=(X_val, s4Val))

Epoch 1/10
188/188 [==============================] - 3s 11ms/step - loss: 2.1434 - accuracy: 0.0492 - val_loss: 1.2337 - val_accuracy: 0.1730
Epoch 2/10
188/188 [==============================] - 2s 8ms/step - loss: 1.3028 - accuracy: 0.1711 - val_loss: 0.7762 - val_accuracy: 0.2561
Epoch 3/10
188/188 [==============================] - 2s 10ms/step - loss: 1.0305 - accuracy: 0.2316 - val_loss: 0.6440 - val_accuracy: 0.3099
Epoch 4/10
188/188 [==============================] - 2s 10ms/step - loss: 0.9530 - accuracy: 0.2431 - val_loss: 0.6111 - val_accuracy: 0.3196
Epoch 5/10
188/188 [==============================] - 2s 11ms/step - loss: 0.9215 - accuracy: 0.2480 - val_loss: 0.5882 - val_accuracy: 0.3078
Epoch 6/10
188/188 [==============================] - 2s 10ms/step - loss: 0.8874 - accuracy: 0.2443 - val_loss: 0.5763 - val_accuracy: 0.3107
Epoch 7/10
188/188 [==============================] - 2s 10ms/step - loss: 0.8680 - accuracy: 0.2462 - val_loss: 0.5281 - val_accuracy: 0.3385


## **4 Students**

In [31]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output
output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)
mm4=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm4.get_layer('d1').set_weights(my_weights)

(None, 256)


In [32]:
for l in mm4.layers[:len(mm4.layers)-1]:
    l.trainable=False

mm4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [33]:
batch_size = 128
mm2_history=mm4.fit([X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1, validation_data=([X_val,X_val,X_val,X_val], Y_val))

Epoch 1/5
375/375 [==============================] - 7s 18ms/step - loss: 0.2656 - accuracy: 0.9120 - val_loss: 0.0661 - val_accuracy: 0.9808
Epoch 2/5
375/375 [==============================] - 5s 14ms/step - loss: 0.2279 - accuracy: 0.9272 - val_loss: 0.0659 - val_accuracy: 0.9814
Epoch 3/5
375/375 [==============================] - 5s 14ms/step - loss: 0.2140 - accuracy: 0.9332 - val_loss: 0.0651 - val_accuracy: 0.9812
Epoch 4/5
375/375 [==============================] - 6s 16ms/step - loss: 0.2085 - accuracy: 0.9345 - val_loss: 0.0659 - val_accuracy: 0.9803
Epoch 5/5
375/375 [==============================] - 5s 13ms/step - loss: 0.2043 - accuracy: 0.9366 - val_loss: 0.0637 - val_accuracy: 0.9818


In [34]:
loss, acc = mm4.evaluate([X_test,X_test,X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 2s 6ms/step - loss: 0.0566 - accuracy: 0.9842


(0.056572265923023224, 0.9842000007629395)

In [21]:
s1.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 4, 4, 16)         

In [35]:
mm4.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
____________________________________________________________________________________________